In [1]:
import import_ipynb
import numpy as np
from GridActions_test3 import GridActions

class GridRewardParams:
    def __init__(self):
        self.boundary_penalty = 1.0 # No need to modify 
        self.empty_battery_penalty = 300.0 # No need to modify 
        self.movement_penalty = 0.1 # No need to modify due to self.movement_range in BaseGrid.ipynb
        #self.tough_penalty = 0.2 # Need to be modified!!!!!!!!!!!!!
        #self.tough_penalty = [0.15, 0.3, 1.0]
        self.tough_penalty = [0.05, 0.25] 
        self.tour_advant = 10.0
        self.successful_landed = 100.0 # No need to modify 
        self.initial_area_penalty = 1.0 # No need to modify 
        self.destination_area = 0.6 # No need to modify 
        

class GridRewards:
    def __init__(self, stats):
        self.params = GridRewardParams()
        self.cumulative_reward: float = 0.0 

        #ModelStats 분석 후에 다시 하는 걸로
        stats.add_log_data_callback('cumulative_reward', self.get_cumulative_reward)
        
#       def add_log_data_callback(self, name: str, callback: callable):
#           self.log_value_callbacks.append((name, callback))
#       self.log_value_callbacks = []

    def get_cumulative_reward(self):
        return self.cumulative_reward

    def calculate_motion_rewards(self, state, action: GridActions, next_state):
        reward = 0.0
        if not next_state.landed: #next_state가 착륙이 아니라면 class state의 landed 함수는 착륙시 True 리턴, @property
            # Penalize battery Consumption
            reward -= self.params.movement_penalty

        if state.position == next_state.position and not next_state.landed and not action == GridActions.HOVER:
            reward -= self.params.boundary_penalty
            #print('self.params.boundary_penalty = ', self.params.boundary_penalty)

        # Penalize battery dead
        if next_state.movement_budget == 0 and not next_state.landed:
            reward -= self.params.empty_battery_penalty
            #print('self.params.empty_battery_penalty =', self.params.empty_battery_penalty)

        # Penalize if tough area
        if next_state.toughed:
            tough_penaltys = np.random.choice(self.params.tough_penalty)
            reward -= tough_penaltys
            
        # Advantage if tour place    
        if next_state.tour_place:
            reward += self.params.tour_advant
                     
        # Penalize if hovering within initial area    
        if next_state.initial_area_18():
            if next_state.initial_area_2():
                #print('area2')
                reward -= self.params.initial_area_penalty
            else :
                #print('area8')
                reward -= self.params.initial_area_penalty * 0.8
        
        if next_state.destination_area_32():
            if next_state.destination_area_8():
                if next_state.destination_area_2():
                    reward += self.params.destination_area
                else :
                    reward += self.params.destination_area * 0.8
            else :
                reward += self.params.destination_area * 0.6
            #print('destination_area')
            
                       
        if next_state.all_landed:
            #print('reward += self.params.successful_landed')
            reward += self.params.successful_landed
            
            
        return reward

    def reset(self):
        self.cumulative_reward = 0

importing Jupyter notebook from GridActions_test2.ipynb
